In [ ]:
####################################################################################################

#    Copyright 2019 Srijan Verma and EMBL-European Bioinformatics Institute

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#        http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

####################################################################################################

In [165]:
import pandas as pd
df_ens_transcript_data = pd.read_csv('/dataset/Ensembl_numeric/all_lincRNA_data/all_lincRNA_sorted_chr/all_lincrna_transcript_data_with_sequences_sorted_chr.csv', index_col=0)

In [166]:
df_ens_transcript_data.head(3)

,SNO,Gene ID,Transcript ID,Biotype,Strand,Seq region Name,Transcript Start,Transcript End,Transcript Length,No. of Exons,Length of Sequences,| Seq_len - Trans_len |,Match ?,Sequences
0,4916,ENSG00000236423,ENST00000413332,lincRNA,1,1,3900372,3905532,634,3,634,0,YES,GCGAAGGTCACAGGCCTTGGGCCAGTGGTGGGACACTGGAGCAGGA...
1,11412,ENSG00000270035,ENST00000602973,lincRNA,-1,1,7698303,7698872,570,1,570,0,YES,GGCACAGGGGAGGCCCTGAGTCACACGATGGTGATGCCCAGGGCAG...
2,11417,ENSG00000270066,ENST00000602755,lincRNA,1,1,109100193,109100619,427,1,427,0,YES,GTTTTAGGGAGGGAGAGCGGCCTGGGTCCTGGGTGTTGTGTGCGGA...


In [167]:
ens_all_seq = df_ens_transcript_data['Sequences'].tolist()

In [168]:
# ens_all_seq[0]

In [169]:
len(df_ens_transcript_data)

13666

In [170]:
from Bio import Seq
import regex as re
from tqdm import tqdm

In [171]:
no_of_amino_acids = []
len_longest_orf = []
longest_orf_seq = []
start_seq_index = []
protein_seq_encoded = []

In [172]:
# no_of_amino_acids

In [173]:
%%time
def get_all_orf_info(input_seq):
    startP = re.compile('ATG')
    nuc = input_seq.replace('\n','')
    longest = (0,)
    for m in startP.finditer(nuc, overlapped=True):
        if len(Seq.Seq(nuc)[m.start():].translate(to_stop=True)) > longest[0]:
            pro = Seq.Seq(nuc)[m.start():].translate(to_stop=True)
            longest = (len(pro), 
                       m.start(), 
                       str(pro),
                       nuc[m.start():m.start()+len(pro)*3+3])
            return(longest)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 10 µs


In [174]:
for i in tqdm(range(len(ens_all_seq))):
    longest = get_all_orf_info(ens_all_seq[i])
    
    if longest == None or longest[0] == 0:
        no_of_amino_acids.append('No start codon found')
        len_longest_orf.append('No start codon found')
        start_seq_index.append('No start codon found')
        protein_seq_encoded.append('No start codon found')
        longest_orf_seq.append('No start codon found')
        continue
        
    else:
        
        no_of_amino_acids.append(longest[0])
        len_longest_orf.append((len(longest[3])-3))
        start_seq_index.append(longest[1])
        protein_seq_encoded.append(longest[2])
        longest_orf_seq.append(longest[3])

  0%|          | 0/13666 [00:00<?, ?it/s]/site-packages/Bio/Seq.py:2609: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)
100%|██████████| 13666/13666 [00:05<00:00, 2394.18it/s]


In [175]:
stop_codon = []

In [176]:
longest_orf_seq[0][-3:]

'TAG'

In [177]:
for i in tqdm(range(len(longest_orf_seq))):
    if longest_orf_seq[i] == 'No start codon found':
        stop_codon.append('No start codon found')
        continue
        
    else:
        if longest_orf_seq[i][-3:] == 'TAA' or longest_orf_seq[i][-3:] == 'TAG' or longest_orf_seq[i][-3:] == 'TGA':
            stop_codon.append('YES')
        else:
            stop_codon.append('NO')

100%|██████████| 13666/13666 [00:00<00:00, 646420.05it/s]


In [178]:
len(stop_codon)

13666

In [179]:
c = 0
index = []
for i in range(len(stop_codon)):
    if stop_codon[i] == 'NO':
        c = c + 1
        index.append(i)
        

In [180]:
c

123

In [181]:
orf_multiple_of_3 = []

In [182]:
# longest_orf_seq

In [183]:
c = 0
index = []
for i in range(len(stop_codon)):
    if len_longest_orf[i] == 'No start codon found':
        orf_multiple_of_3.append('No start codon found')
        continue
        
    else:
        if len_longest_orf[i] % 3 == 0:
            orf_multiple_of_3.append('YES')
        else:
            orf_multiple_of_3.append('NO')
            c = c + 1
            index.append(i)
        

In [184]:
c

94

In [185]:
# no_of_amino_acids = []
# len_longest_orf = []
# longest_orf_seq = []
# start_seq_index = []
# protein_seq_encoded = []
# orf_multiple_of_3 = []
# stop_codon = []

In [186]:
# orf_multiple_of_3

In [187]:
df_ens_transcript_data['Len of longest ORF'] = len_longest_orf
df_ens_transcript_data['No. of Amino Acids'] = no_of_amino_acids
df_ens_transcript_data['Longest ORF Seq'] = longest_orf_seq
df_ens_transcript_data['ORF Start Index for longest Seq'] = start_seq_index
df_ens_transcript_data['Protein Seq encoded'] = protein_seq_encoded
df_ens_transcript_data['Stop codon present ?'] = stop_codon
df_ens_transcript_data['ORF multiple of 3 ?'] = orf_multiple_of_3

In [188]:
df_ens_transcript_data.head(3)

,SNO,Gene ID,Transcript ID,Biotype,Strand,Seq region Name,Transcript Start,Transcript End,Transcript Length,No. of Exons,...,| Seq_len - Trans_len |,Match ?,Sequences,Len of longest ORF,No. of Amino Acids,Longest ORF Seq,ORF Start Index for longest Seq,Protein Seq encoded,Stop codon present ?,ORF multiple of 3 ?
0,4916,ENSG00000236423,ENST00000413332,lincRNA,1,1,3900372,3905532,634,3,...,0,YES,GCGAAGGTCACAGGCCTTGGGCCAGTGGTGGGACACTGGAGCAGGA...,33,11,ATGTTCATGAGGGCGCGTCCCTTGGCACCCCTGTAG,65,MFMRARPLAPL,YES,YES
1,11412,ENSG00000270035,ENST00000602973,lincRNA,-1,1,7698303,7698872,570,1,...,0,YES,GGCACAGGGGAGGCCCTGAGTCACACGATGGTGATGCCCAGGGCAG...,42,14,ATGGTGATGCCCAGGGCAGAAGATGTCAGAAGACAGGCAGTTTGA,27,MVMPRAEDVRRQAV,YES,YES
2,11417,ENSG00000270066,ENST00000602755,lincRNA,1,1,109100193,109100619,427,1,...,0,YES,GTTTTAGGGAGGGAGAGCGGCCTGGGTCCTGGGTGTTGTGTGCGGA...,138,46,ATGAAAAGCTCTCCTCCTGCGAAGCCCCCTCGGGGCGCTGTGCAGC...,227,MKSSPPAKPPRGAVQRGPLGGGHAVVPEVPELALRGPALRSDELIR,YES,YES


In [189]:
df_ens_transcript_data.iloc[304]

SNO                                                                             3803
Gene ID                                                              ENSG00000232453
Transcript ID                                                        ENST00000427292
Biotype                                                                      lincRNA
Strand                                                                            -1
Seq region Name                                                                    1
Transcript Start                                                            58895845
Transcript End                                                              58896703
Transcript Length                                                                219
No. of Exons                                                                       2
Length of Sequences                                                              219
| Seq_len - Trans_len |                                          

In [190]:
df_ens_transcript_data.to_csv('/feature_selection/ORF/Ensembl/all_lincrna_transcript_data_orf_info.csv')

In [1]:
import pandas as pd
df_ens_transcript_data = pd.read_csv('/1.lncRNA/feature_selection/ORF/Ensembl/all_lincrna_transcript_data_orf_info.csv', index_col=0)

In [2]:
df_ens_transcript_data.head(3)

,SNO,Gene ID,Transcript ID,Biotype,Strand,Seq region Name,Transcript Start,Transcript End,Transcript Length,No. of Exons,...,Match ?,Sequences,GC %,Len of longest ORF,No. of Amino Acids,Longest ORF Seq,ORF Start Index for longest Seq,Protein Seq encoded,Stop codon present ?,ORF multiple of 3 ?
0,4916,ENSG00000236423,ENST00000413332,lincRNA,1,1,3900372,3905532,634,3,...,YES,GCGAAGGTCACAGGCCTTGGGCCAGTGGTGGGACACTGGAGCAGGA...,59.31,33,11,ATGTTCATGAGGGCGCGTCCCTTGGCACCCCTGTAG,65,MFMRARPLAPL,YES,YES
1,11412,ENSG00000270035,ENST00000602973,lincRNA,-1,1,7698303,7698872,570,1,...,YES,GGCACAGGGGAGGCCCTGAGTCACACGATGGTGATGCCCAGGGCAG...,55.26,42,14,ATGGTGATGCCCAGGGCAGAAGATGTCAGAAGACAGGCAGTTTGA,27,MVMPRAEDVRRQAV,YES,YES
2,11417,ENSG00000270066,ENST00000602755,lincRNA,1,1,109100193,109100619,427,1,...,YES,GTTTTAGGGAGGGAGAGCGGCCTGGGTCCTGGGTGTTGTGTGCGGA...,65.81,138,46,ATGAAAAGCTCTCCTCCTGCGAAGCCCCCTCGGGGCGCTGTGCAGC...,227,MKSSPPAKPPRGAVQRGPLGGGHAVVPEVPELALRGPALRSDELIR,YES,YES


In [3]:
from tqdm import tqdm

In [4]:
c = 0
index = []
for i in tqdm(range(len(df_ens_transcript_data))):
    if df_ens_transcript_data['Stop codon present ?'][i] == 'YES' and df_ens_transcript_data['ORF multiple of 3 ?'][i] == 'NO':
        c = c + 1
        index.append(i)

100%|██████████| 13666/13666 [00:00<00:00, 29221.83it/s]


In [5]:
c

7

In [6]:
index

[776, 1856, 4745, 7076, 8135, 10276, 11922]

In [7]:
df_ens_transcript_data.iloc[776]

SNO                                                                             1384
Gene ID                                                              ENSG00000224687
Transcript ID                                                        ENST00000452867
Biotype                                                                      lincRNA
Strand                                                                            -1
Seq region Name                                                                    1
Transcript Start                                                           178093150
Transcript End                                                             178093572
Transcript Length                                                                300
No. of Exons                                                                       2
Length of Sequences                                                              300
| Seq_len - Trans_len |                                          

In [10]:
from Bio.SeqUtils import GC123
from tqdm import tqdm

In [11]:
gc_percentage = []
nan_values = 0
nan_index = []

In [12]:
%%time
def get_gc_percentage():
    for i in tqdm(range(len(df_ens_transcript_data['Sequences']))):
        
        if type(df_ens_transcript_data['Sequences'][i]) is float:
            gc_percentage.append('Nan')
            df_ens_transcript_data['Sequences'][i] = 'Nan'
            nan_values = nan_values + 1
            nan_index.append(i)
            
        else: 
            gc_percentage.append(GC123(df_ens_transcript_data['Sequences'][i])[0])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.72 µs


In [13]:
%%time
get_gc_percentage()

100%|██████████| 13666/13666 [00:31<00:00, 429.19it/s]

CPU times: user 24.9 s, sys: 632 ms, total: 25.5 s
Wall time: 31.8 s


In [14]:
nan_values

0

In [25]:
for i in tqdm(range(len(gc_percentage))):
    gc_percentage[i] = float("{0:.2f}".format(gc_percentage[i]))

100%|██████████| 13666/13666 [00:00<00:00, 480500.28it/s]


In [27]:
gc_percentage[0:5]

[59.31, 55.26, 65.81, 51.52, 50.85]

In [29]:
a = list(df_ens_transcript_data.columns.values)

In [31]:
b = a[0:14]

In [33]:
b.append('GC %')

In [35]:
df_ens_transcript_data['GC %'] = gc_percentage

In [36]:
c = b + a[14:]

In [40]:
# c

In [38]:
df_ens_transcript_data = df_ens_transcript_data[c]

In [39]:
df_ens_transcript_data.head(3)

,SNO,Gene ID,Transcript ID,Biotype,Strand,Seq region Name,Transcript Start,Transcript End,Transcript Length,No. of Exons,...,Match ?,Sequences,GC %,Len of longest ORF,No. of Amino Acids,Longest ORF Seq,ORF Start Index for longest Seq,Protein Seq encoded,Stop codon present ?,ORF multiple of 3 ?
0,4916,ENSG00000236423,ENST00000413332,lincRNA,1,1,3900372,3905532,634,3,...,YES,GCGAAGGTCACAGGCCTTGGGCCAGTGGTGGGACACTGGAGCAGGA...,59.31,33,11,ATGTTCATGAGGGCGCGTCCCTTGGCACCCCTGTAG,65,MFMRARPLAPL,YES,YES
1,11412,ENSG00000270035,ENST00000602973,lincRNA,-1,1,7698303,7698872,570,1,...,YES,GGCACAGGGGAGGCCCTGAGTCACACGATGGTGATGCCCAGGGCAG...,55.26,42,14,ATGGTGATGCCCAGGGCAGAAGATGTCAGAAGACAGGCAGTTTGA,27,MVMPRAEDVRRQAV,YES,YES
2,11417,ENSG00000270066,ENST00000602755,lincRNA,1,1,109100193,109100619,427,1,...,YES,GTTTTAGGGAGGGAGAGCGGCCTGGGTCCTGGGTGTTGTGTGCGGA...,65.81,138,46,ATGAAAAGCTCTCCTCCTGCGAAGCCCCCTCGGGGCGCTGTGCAGC...,227,MKSSPPAKPPRGAVQRGPLGGGHAVVPEVPELALRGPALRSDELIR,YES,YES


In [41]:
df_ens_transcript_data.iloc[0]

SNO                                                                             4916
Gene ID                                                              ENSG00000236423
Transcript ID                                                        ENST00000413332
Biotype                                                                      lincRNA
Strand                                                                             1
Seq region Name                                                                    1
Transcript Start                                                             3900372
Transcript End                                                               3905532
Transcript Length                                                                634
No. of Exons                                                                       3
Length of Sequences                                                              634
| Seq_len - Trans_len |                                          

In [42]:
len(c)

22

In [43]:
df_ens_transcript_data.to_csv('/1.lncRNA/feature_selection/ORF/Ensembl/all_lincrna_transcript_data_orf_info.csv')